In [1]:
# STEP 0: QEPC project bootstrap (works from any notebook folder)

import sys
from pathlib import Path

def find_project_root() -> Path:
    """
    Walk upwards from the current working directory until we find
    a folder that looks like the QEPC project root
    (has main.py and a qepc/ package).
    """
    here = Path.cwd()
    for parent in [here] + list(here.parents):
        if (parent / "main.py").exists() and (parent / "qepc").exists():
            return parent
    # Fallback: just use cwd
    return here

PROJECT_ROOT = find_project_root()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("QEPC project root:", PROJECT_ROOT)

from qepc_autoload import qepc_step
import pandas as pd

qepc_step("QEPC smoketest notebook started")


QEPC project root: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project
[QEPC Paths] Project Root set: /home/2dbcc135-5358-4730-8441-82ada9ea8087/qepc_project
[QEPC] Autoload complete.


In [2]:
# STEP 1: Import strengths, lambda engine, simulator

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation

qepc_step("Imported strengths_v2, lambda_engine, and simulator successfully")


In [3]:
# STEP 2: Calculate team strengths from Team_Stats

strengths_df = calculate_advanced_strengths(
    cutoff_date=None,   # or "2025-11-01" etc if you want a backtest-style cutoff
    verbose=True,
)

display(strengths_df.head())
print(f"Total teams in strengths_df: {len(strengths_df)}")


Built team strengths (cutoff: 2025-11-28)
  Teams: 30
  ORtg range: 109.4 – 126.1
  DRtg range: 106.0 – 127.6
  Pace range: 111.8 – 122.9
  Volatility range: 6.6 – 18.8


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,118.266584,114.448783,116.357684,10.756586
1,Boston Celtics,114.814485,108.748201,111.781343,12.000244
2,Brooklyn Nets,111.445684,119.531108,115.488396,9.178288
3,Charlotte Hornets,116.326682,119.407084,117.866883,12.668467
4,Chicago Bulls,122.025690,122.855327,122.440508,10.225093


Total teams in strengths_df: 30


In [4]:
# STEP 3: Create a tiny schedule with 1 test game

qepc_step("Building a tiny test schedule")

# Pick first two teams
home_team = strengths_df.iloc[0]["Team"]
away_team = strengths_df.iloc[1]["Team"]

print("Home team:", home_team)
print("Away team:", away_team)

schedule_df = pd.DataFrame(
    {
        "Home Team": [home_team],
        "Away Team": [away_team],
        # Optional situational columns (we'll leave them out for now):
        # "rest_days_home": [2],
        # "rest_days_away": [1],
        # "home_b2b": [False],
        # "away_b2b": [False],
    }
)

display(schedule_df)


Home team: Atlanta Hawks
Away team: Boston Celtics


,Home Team,Away Team
0,Atlanta Hawks,Boston Celtics


In [5]:
# STEP 4: Compute lambda for the test game

qepc_step("Computing lambda for test game")

lambda_df = compute_lambda(
    schedule_df=schedule_df,
    team_stats_df=strengths_df,
    include_situational=False,  # True if you add rest_days_* & *_b2b columns
)

display(lambda_df[["Home Team", "Away Team", "lambda_home", "lambda_away", "vol_home", "vol_away"]])


[QEPC Lambda] Computed λ for 1/1 games.


,Home Team,Away Team,lambda_home,lambda_away,vol_home,vol_away
0,Atlanta Hawks,Boston Celtics,105.196059,104.551454,10.756586,12.000244


In [6]:
# STEP 5: Simulate the game

qepc_step("Running QEPC simulation")

sim_results = run_qepc_simulation(
    df=lambda_df,
    num_trials=5000,      # enough for a stable feel but not too slow
    use_normal=True,      # NBA-friendly
    correlated_scores=True,
)

display(
    sim_results[
        [
            "Home Team",
            "Away Team",
            "lambda_home",
            "lambda_away",
            "Sim_Home_Score",
            "Sim_Away_Score",
            "Home_Win_Prob",
            "Away_Win_Prob",
            "Expected_Spread",
            "Expected_Score_Total",
        ]
    ]
)


[QEPC Simulator] Running 5000 trials (Normal, Correlated)...
[QEPC Simulator] Simulation complete.


,Home Team,Away Team,lambda_home,lambda_away,Sim_Home_Score,Sim_Away_Score,Home_Win_Prob,Away_Win_Prob,Expected_Spread,Expected_Score_Total
0,Atlanta Hawks,Boston Celtics,105.196059,104.551454,105.167665,104.495092,0.5156,0.4844,0.672573,209.662758
